In [2]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf

rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# set Dropout
do = 0.3

if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# feat = feat[mask]
# API model, if use RNN, first two rnn layer must return_sequences=True


In [3]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(dropout_rate)(net_speech)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_speech) for name in target_names]

    model = Model(input_speech, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(0.1, 0.5, 0.4)
#model = api_model(1, 1, 1)
#model.summary()
# main function for for LOSO
# main function for for LOSO


In [4]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:7869]],
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[7869:]], vad[7869:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_7layer.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 6s 980us/step - loss: 1.8801 - v_loss: 0.9152 - a_loss: 0.4223 - d_loss: 0.5419 - v_ccc: 0.0844 - a_ccc: 0.5775 - d_ccc: 0.4580 - val_loss: 2.4912 - val_v_loss: 0.8546 - val_a_loss: 0.8352 - val_d_loss: 0.7963 - val_v_ccc: 0.1446 - val_a_ccc: 0.1645 - val_d_ccc: 0.1997
Epoch 2/50
6295/6295 [==============================] - 3s 533us/step - loss: 1.5900 - v_loss: 0.7467 - a_loss: 0.3702 - d_loss: 0.4730 - v_ccc: 0.2534 - a_ccc: 0.6296 - d_ccc: 0.5270 - val_loss: 2.1293 - val_v_loss: 0.8557 - val_a_loss: 0.6238 - val_d_loss: 0.6529 - val_v_ccc: 0.1443 - val_a_ccc: 0.3797 - val_d_ccc: 0.3468
Epoch 3/50
6295/6295 [==============================] - 3s 510us/step - loss: 1.5607 - v_loss: 0.7088 - a_loss: 0.3748 - d_loss: 0.4772 - v_ccc: 0.2913 - a_ccc: 0.6252 - d_ccc: 0.5228 - val_loss: 1.9503 - val_v_loss: 0

Epoch 11/50
6295/6295 [==============================] - 4s 564us/step - loss: 1.4349 - v_loss: 0.6267 - a_loss: 0.3544 - d_loss: 0.4543 - v_ccc: 0.3734 - a_ccc: 0.6460 - d_ccc: 0.5458 - val_loss: 2.0009 - val_v_loss: 0.8498 - val_a_loss: 0.5314 - val_d_loss: 0.6168 - val_v_ccc: 0.1486 - val_a_ccc: 0.4697 - val_d_ccc: 0.3807
Epoch 12/50
6295/6295 [==============================] - 3s 554us/step - loss: 1.4283 - v_loss: 0.6132 - a_loss: 0.3565 - d_loss: 0.4584 - v_ccc: 0.3866 - a_ccc: 0.6435 - d_ccc: 0.5416 - val_loss: 1.9846 - val_v_loss: 0.8437 - val_a_loss: 0.5211 - val_d_loss: 0.6167 - val_v_ccc: 0.1517 - val_a_ccc: 0.4811 - val_d_ccc: 0.3825
Epoch 13/50
6295/6295 [==============================] - 4s 572us/step - loss: 1.4123 - v_loss: 0.6043 - a_loss: 0.3493 - d_loss: 0.4593 - v_ccc: 0.3960 - a_ccc: 0.6511 - d_ccc: 0.5406 - val_loss: 1.9985 - val_v_loss: 0.8256 - val_a_loss: 0.5576 - val_d_loss: 0.6059 - val_v_ccc: 0.1694 - val_a_ccc: 0.4433 - val_d_ccc: 0.3889
Epoch 14/50
2170/21

Epoch 22/50
6295/6295 [==============================] - 4s 566us/step - loss: 1.1669 - v_loss: 0.4382 - a_loss: 0.3078 - d_loss: 0.4213 - v_ccc: 0.5622 - a_ccc: 0.6922 - d_ccc: 0.5787 - val_loss: 1.8556 - val_v_loss: 0.8067 - val_a_loss: 0.4637 - val_d_loss: 0.5878 - val_v_ccc: 0.1933 - val_a_ccc: 0.5370 - val_d_ccc: 0.4140
Epoch 23/50
6295/6295 [==============================] - 3s 548us/step - loss: 1.1484 - v_loss: 0.4331 - a_loss: 0.3016 - d_loss: 0.4145 - v_ccc: 0.5671 - a_ccc: 0.6988 - d_ccc: 0.5856 - val_loss: 1.9638 - val_v_loss: 0.8221 - val_a_loss: 0.5165 - val_d_loss: 0.6271 - val_v_ccc: 0.1738 - val_a_ccc: 0.4866 - val_d_ccc: 0.3758
Epoch 24/50
6295/6295 [==============================] - 3s 555us/step - loss: 1.1256 - v_loss: 0.4198 - a_loss: 0.3013 - d_loss: 0.4047 - v_ccc: 0.5806 - a_ccc: 0.6987 - d_ccc: 0.5951 - val_loss: 1.9147 - val_v_loss: 0.8692 - val_a_loss: 0.4557 - val_d_loss: 0.5882 - val_v_ccc: 0.1310 - val_a_ccc: 0.5417 - val_d_ccc: 0.4126
Epoch 25/50
6295/62

2170/2170 [==============================] - 0s 147us/step
[0.09925832599401474, 0.5639711618423462, 0.42508402466773987]
3
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 7s 1ms/step - loss: 1.8498 - v_loss: 0.8801 - a_loss: 0.4329 - d_loss: 0.5376 - v_ccc: 0.1200 - a_ccc: 0.5673 - d_ccc: 0.4630 - val_loss: 2.6718 - val_v_loss: 0.9238 - val_a_loss: 0.8849 - val_d_loss: 0.8575 - val_v_ccc: 0.0745 - val_a_ccc: 0.1152 - val_d_ccc: 0.1385
Epoch 2/50
6295/6295 [==============================] - 3s 518us/step - loss: 1.6003 - v_loss: 0.7561 - a_loss: 0.3699 - d_loss: 0.4745 - v_ccc: 0.2442 - a_ccc: 0.6299 - d_ccc: 0.5256 - val_loss: 2.0201 - val_v_loss: 0.8638 - val_a_loss: 0.5746 - val_d_loss: 0.5880 - val_v_ccc: 0.1363 - val_a_ccc: 0.4305 - val_d_ccc: 0.4131
Epoch 3/50
6295/6295 [==============================] - 3s 550us/step - loss: 1.5425 - v_loss: 0.7164 - a_loss: 0.3634 - d_loss: 0.4629 - v_ccc: 0.2840 - a_ccc: 0.6367 - d_ccc: 0

6295/6295 [==============================] - 4s 577us/step - loss: 1.4304 - v_loss: 0.6112 - a_loss: 0.3612 - d_loss: 0.4581 - v_ccc: 0.3887 - a_ccc: 0.6391 - d_ccc: 0.5418 - val_loss: 2.0161 - val_v_loss: 0.8565 - val_a_loss: 0.5447 - val_d_loss: 0.6155 - val_v_ccc: 0.1424 - val_a_ccc: 0.4567 - val_d_ccc: 0.3847
Epoch 12/50
6295/6295 [==============================] - 3s 524us/step - loss: 1.4262 - v_loss: 0.6124 - a_loss: 0.3545 - d_loss: 0.4597 - v_ccc: 0.3876 - a_ccc: 0.6456 - d_ccc: 0.5407 - val_loss: 2.0163 - val_v_loss: 0.8469 - val_a_loss: 0.5412 - val_d_loss: 0.6254 - val_v_ccc: 0.1493 - val_a_ccc: 0.4588 - val_d_ccc: 0.3756
Epoch 13/50
2170/2170 [==============================] - 0s 154us/step
[0.09415436536073685, 0.5514504313468933, 0.43077710270881653]
5
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 7s 1ms/step - loss: 1.9429 - v_loss: 0.9591 - a_loss: 0.4220 - d_loss: 0.5618 - v_ccc: 0.0412 - a_ccc: 0.5780 - d_ccc:

...

6295/6295 [==============================] - 3s 492us/step - loss: 1.4421 - v_loss: 0.6202 - a_loss: 0.3618 - d_loss: 0.4599 - v_ccc: 0.3799 - a_ccc: 0.6381 - d_ccc: 0.5399 - val_loss: 2.0165 - val_v_loss: 0.8457 - val_a_loss: 0.5524 - val_d_loss: 0.6167 - val_v_ccc: 0.1521 - val_a_ccc: 0.4494 - val_d_ccc: 0.3820
Epoch 12/50
6295/6295 [==============================] - 3s 499us/step - loss: 1.4227 - v_loss: 0.6115 - a_loss: 0.3532 - d_loss: 0.4583 - v_ccc: 0.3887 - a_ccc: 0.6467 - d_ccc: 0.5419 - val_loss: 2.0337 - val_v_loss: 0.8349 - val_a_loss: 0.5667 - val_d_loss: 0.6282 - val_v_ccc: 0.1609 - val_a_ccc: 0.4350 - val_d_ccc: 0.3704
Epoch 13/50
2170/2170 [==============================] - 0s 182us/step
[0.09451503306627274, 0.5566364526748657, 0.4346451759338379]
13
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 9s 1ms/step - loss: 1.8916 - v_loss: 0.9356 - a_loss: 0.4133 - d_loss: 0.5422 - v_ccc: 0.0640 - a_ccc: 0.5868 - d_ccc:

Epoch 23/50
6295/6295 [==============================] - 3s 512us/step - loss: 1.2632 - v_loss: 0.4845 - a_loss: 0.3391 - d_loss: 0.4396 - v_ccc: 0.5156 - a_ccc: 0.6610 - d_ccc: 0.5603 - val_loss: 1.9931 - val_v_loss: 0.8186 - val_a_loss: 0.5252 - val_d_loss: 0.6462 - val_v_ccc: 0.1807 - val_a_ccc: 0.4744 - val_d_ccc: 0.3518
Epoch 24/50
6295/6295 [==============================] - 3s 511us/step - loss: 1.2085 - v_loss: 0.4545 - a_loss: 0.3247 - d_loss: 0.4290 - v_ccc: 0.5453 - a_ccc: 0.6752 - d_ccc: 0.5710 - val_loss: 1.8691 - val_v_loss: 0.7898 - val_a_loss: 0.4746 - val_d_loss: 0.6004 - val_v_ccc: 0.2090 - val_a_ccc: 0.5225 - val_d_ccc: 0.3993
Epoch 25/50
6295/6295 [==============================] - 3s 494us/step - loss: 1.1764 - v_loss: 0.4368 - a_loss: 0.3171 - d_loss: 0.4224 - v_ccc: 0.5632 - a_ccc: 0.6828 - d_ccc: 0.5776 - val_loss: 1.8665 - val_v_loss: 0.7821 - val_a_loss: 0.4757 - val_d_loss: 0.6083 - val_v_ccc: 0.2169 - val_a_ccc: 0.5240 - val_d_ccc: 0.3926
Epoch 26/50
6295/62

Epoch 8/50
6295/6295 [==============================] - 3s 482us/step - loss: 1.4615 - v_loss: 0.6407 - a_loss: 0.3632 - d_loss: 0.4573 - v_ccc: 0.3590 - a_ccc: 0.6367 - d_ccc: 0.5427 - val_loss: 2.0344 - val_v_loss: 0.8517 - val_a_loss: 0.5610 - val_d_loss: 0.6187 - val_v_ccc: 0.1469 - val_a_ccc: 0.4413 - val_d_ccc: 0.3775
Epoch 9/50
6295/6295 [==============================] - 3s 491us/step - loss: 1.4623 - v_loss: 0.6378 - a_loss: 0.3594 - d_loss: 0.4654 - v_ccc: 0.3620 - a_ccc: 0.6408 - d_ccc: 0.5349 - val_loss: 1.9961 - val_v_loss: 0.8525 - val_a_loss: 0.5292 - val_d_loss: 0.6142 - val_v_ccc: 0.1443 - val_a_ccc: 0.4738 - val_d_ccc: 0.3857
Epoch 10/50
6295/6295 [==============================] - 3s 448us/step - loss: 1.4423 - v_loss: 0.6297 - a_loss: 0.3504 - d_loss: 0.4632 - v_ccc: 0.3708 - a_ccc: 0.6498 - d_ccc: 0.5371 - val_loss: 2.0126 - val_v_loss: 0.8508 - val_a_loss: 0.5480 - val_d_loss: 0.6141 - val_v_ccc: 0.1486 - val_a_ccc: 0.4551 - val_d_ccc: 0.3837
Epoch 11/50
6295/6295

6295/6295 [==============================] - 3s 528us/step - loss: 1.5162 - v_loss: 0.6787 - a_loss: 0.3645 - d_loss: 0.4726 - v_ccc: 0.3212 - a_ccc: 0.6353 - d_ccc: 0.5273 - val_loss: 1.9615 - val_v_loss: 0.8681 - val_a_loss: 0.5110 - val_d_loss: 0.5839 - val_v_ccc: 0.1287 - val_a_ccc: 0.4945 - val_d_ccc: 0.4154
Epoch 6/50
6295/6295 [==============================] - 3s 519us/step - loss: 1.4978 - v_loss: 0.6680 - a_loss: 0.3637 - d_loss: 0.4662 - v_ccc: 0.3320 - a_ccc: 0.6364 - d_ccc: 0.5338 - val_loss: 2.0076 - val_v_loss: 0.8543 - val_a_loss: 0.5481 - val_d_loss: 0.6037 - val_v_ccc: 0.1428 - val_a_ccc: 0.4541 - val_d_ccc: 0.3955
Epoch 7/50
6295/6295 [==============================] - 3s 527us/step - loss: 1.4721 - v_loss: 0.6486 - a_loss: 0.3607 - d_loss: 0.4633 - v_ccc: 0.3517 - a_ccc: 0.6394 - d_ccc: 0.5368 - val_loss: 2.0222 - val_v_loss: 0.8632 - val_a_loss: 0.5478 - val_d_loss: 0.6124 - val_v_ccc: 0.1351 - val_a_ccc: 0.4560 - val_d_ccc: 0.3867
Epoch 8/50
6295/6295 [===========

Epoch 15/50
6295/6295 [==============================] - 3s 523us/step - loss: 1.3990 - v_loss: 0.5939 - a_loss: 0.3558 - d_loss: 0.4501 - v_ccc: 0.4064 - a_ccc: 0.6442 - d_ccc: 0.5504 - val_loss: 2.0031 - val_v_loss: 0.8499 - val_a_loss: 0.5502 - val_d_loss: 0.6049 - val_v_ccc: 0.1490 - val_a_ccc: 0.4528 - val_d_ccc: 0.3951
Epoch 16/50
2170/2170 [==============================] - 0s 196us/step
[0.10463037341833115, 0.5630737543106079, 0.4332183003425598]
18
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 11s 2ms/step - loss: 1.9015 - v_loss: 0.9479 - a_loss: 0.4138 - d_loss: 0.5391 - v_ccc: 0.0516 - a_ccc: 0.5860 - d_ccc: 0.4609 - val_loss: 2.5341 - val_v_loss: 0.8566 - val_a_loss: 0.8493 - val_d_loss: 0.8227 - val_v_ccc: 0.1418 - val_a_ccc: 0.1500 - val_d_ccc: 0.1741
Epoch 2/50
6295/6295 [==============================] - 3s 534us/step - loss: 1.5985 - v_loss: 0.7525 - a_loss: 0.3692 - d_loss: 0.4757 - v_ccc: 0.2471 - a_ccc: 0.6